# Выполнено Зверьковой Елизаветой Алексеевной

# Сессия 5

## В данной сессии нам необходимо: 
- реализовать бота
- подготовить документацию

In [1]:
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import warnings
warnings.filterwarnings("ignore")
import re
import numpy as np
import pymorphy2

---

# 4.1 Разработка бота

## На данном этапе необходимо выполнить: 
- разработайтку графического интерфейса, позволяющий пользователю вводить текст обращения, отправлять его на классификатор и получать сообщение о приеме обращения с определением его типа

Для начала разработаем функции для будущего бота

Функция для поиска необходимой информации из файла

In [23]:
def parser(filename):
    result = []

    with open(filename, encoding='utf-8') as f:
        for line in f:
            id_petition = re.search(r'(?<={\"id\":)(?P<id>.*?)(?=\,)', line)
            date = re.search(r'(?<=\"dt\":\")(?P<date>.*?)(?=T)', line)
            text_petition = re.search(r'(?<=\"body\":\[{\"text\":\")(?P<petition1>.*?)(?=\")|(?<=\,{\"text\":\")(?P<petition2>.*?)(?=\.)', line)
            reason = re.search(r'(?<=\"reason\":{\"id\":)(.*?,)(\"name\":\")(?P<reason_p>.*?)(?=\")', line)
            category = re.search(r'(?<=\"category\":{\"id\":)(.*?,)(\"name\":\")(?P<category_p>.*?)(?=\")', line)
        
        result.append([
            id_petition.group(1),
            date.group(1),
            text_petition.group(1),
            reason.group(3),
            category.group(3)
            

    return ''.join(result)

SyntaxError: invalid syntax (1046338362.py, line 20)

Функция для очистки слов

In [2]:
def cleaning(words):
    words = re.sub('[^а-яА-ЯёЁ]', ' ', words).lower().strip()
    stopwords = ["тд","c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]
    words = ' '.join([word for word in words.split() if word not in (stopwords)])
    morph = pymorphy2.MorphAnalyzer()
    words = " ".join(morph.parse(word)[0].normal_form for word in words.split())
    return words

Функция для загрузки модели

In [3]:
def load_model(filename = 'model.pkl'):
    with open(filename, 'rb') as file:
        model = pickle.load(file)
    return model

In [4]:
#загрузка модели
model = load_model()

In [5]:
model

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', KNeighborsClassifier())])

Функция для определения категории обращения

In [6]:
def make_pred(words, model, p_num=1):
    if p_num==1:
        pred = model.predict([words])
        return pred
    else:
        allpreds = model.predict_proba([words]).reshape(-1)
        R =[]
        for i in allpreds.argsort()[-p_num:][::-1]:
            R.append(model.classes_[i])
        return R

Функция для информационной справки

In [9]:
def helpp():
    print("Bot: Если у вас возникли какие-либо проблемы, убедитесь что все ваши файлы находятся в одной директории и вы везде указываете расширение файлов. В директрории должен обязательно содержаться файл model.pkl. Если вы считаете что вам выдается неточный результат знайте, что чем больше вы расскажете или напишите в вашем резюме о себе, тeм точнее будет результат. Eсли у вас есть обученная модель в формате pkl то вы можете заменить файл model.pkl в директории на свой \n\nВ боте содержaтся следующие функции load_model(filename = 'model.pkl'),pdf_text(filename), cleaning(words), make_pred(words, model, p_num=1), helpp()")

Функция для ответа бота

In [7]:
def chatbot_response(msg):
    if msg == 'нрпто':
        return 'Нет подключения к интернету'
    if any(word in msg.lower() for word in ("умеешь", "команды")):
        # Get text types
        intros = bb.intros(language="ru")

        # Get the first text type
        intro = next(intros)
        res = bb.balaboba(
            f'Вопрос: {msg}. Ответ:', intro=intro.number
        ).split('\n')[0]
    
    
    
    if any(word in msg.lower() for word in ("жалобу", 'обращение')):
        return 'Напишите вашу жалобу или обращение, начиная со слов "У меня..."'
    if any(word in msg.lower() for word in ("у меня")):
        clean = cleaning(msg)
        pred = make_pred(clean, model, p_num=1)
        res = f'Ваша жалоба относится к категории {pred[0]}. Обратитесь туда.'

Функция для отправки сообщения боту

In [8]:
def send():
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

In [11]:
from balaboba import Balaboba

bb = Balaboba()

# Get text types
intros = bb.intros(language="ru")

# Get the first text type
intro = next(intros)

# Print Balaboba's response to the "Hello" query
response = bb.balaboba("тутуту", intro=intro.number)
print(response)

тутуту ту
Туту-ту-та
Ту-ту, ту-ту
ТУТУ-ТУТУ
Тутуту туту
Туруру-ру
Туру-туру-туру
ТУРУРУ ТУ
ТУ-РУ-РУ
ТУУ-ТУУ
ТУТ-ТУТ
ТУ ТУ ТУТУ
ТУТУТУ


## Вывод:
Здесь мы реализовали все необходимые функции ждя работы нашего бота

# 5.2. Реализация бота

In [31]:
import balaboba
from tkinter.ttk import *
from tkinter.filedialog import askopenfile 
import time

def chatbot_response(msg):
    if msg == 'нрпто':
        return 'Нет подключения к интернету'
    if any(word in msg.lower() for word in ("умеешь", "команды")):
        # Get text types
        intros = bb.intros(language="ru")

        # Get the first text type
        intro = next(intros)
        res = bb.balaboba(
            f'Вопрос: {msg}. Ответ:', intro=intro.number
        ).split('\n')[0]

    elif any(word in msg.lower() for word in ("жалобу", 'обращение')):
        return 'Напишите вашу жалобу или обращение, начиная со слов "У меня..."'
    elif any(word in msg.lower() for word in ("меня")):
        clean = cleaning(msg)
        pred = make_pred(clean, model, p_num=1)
        res = f'Ваша жалоба относится к категории {pred[0]}. Обратитесь туда.'
    else:
        return "Повторите ваш запрос"
    
        
    return res



# Creating GUI with tkinter
import tkinter as tk
from tkinter import *



base = tk.Tk()
base.title("Колледж связи 54")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)







#функция для открытия файла
def open_file():
    file_path = askopenfile(mode='r', filetypes=[('Image Files', '*jpeg')])
    if file_path is not None:
        pass

def uploadFiles():
    pb1 = Progressbar(
        ws, 
        orient=HORIZONTAL, 
        length=300, 
        mode='determinate'
        )
    pb1.grid(row=4, columnspan=3, pady=20)
    for i in range(5):
        ws.update_idletasks()
        pb1['value'] += 20
        time.sleep(1)
    pb1.destroy()
    Label(ws, text='File Uploaded Successfully!', foreground='green').grid(row=4, columnspan=3, pady=10)
        
    
    
FileB = Label( 
    text='Upload in jpg format '
    )
FileB.grid(row=0, column=0, padx=10)

FileButton = Button(
    text ='Choose File', 
    command = lambda:open_file()
    ) 
FileButton.grid(row=0, column=1)

upld = Button(
     
    text='Upload Files', 
    command=uploadFiles
    )
upld.grid(row=3, columnspan=3, pady=10)  






def send():
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


# Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial")
ChatLog.config(state=DISABLED)

# Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

# Create Button to send message
SendButton = Button(base,
                    font=("Verdana", 12, 'bold'),
                    text="Send",
                    width="12",
                    height=5,
                    bd=0,
                    bg="#32de97",
                    activebackground="#3c9d9b",
                    fg='#ffffff',
                    command=send,
                    cursor="heart")

# Create the box to enter message
EntryBox = Text(base, bd=0, bg="white", width="29", height="5", font="Roboto")
# EntryBox.bind("<Return>", send)

# Place all components on the screen
scrollbar.place(x=376, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

ChatLog.config(state=NORMAL)
ChatLog.insert(
    END, "Bot: " +
    'Здравствуйте, Вас приветсвует бот жалоб и обращений.\n Для определения категории жалобы, введите "подать жалобу/обращение" '
    + '\n\n')



base.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\zverk\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\zverk\AppData\Local\Temp\ipykernel_15344\2172172347.py", line 58, in uploadFiles
    ws,
NameError: name 'ws' is not defined


Сейчас нам необходимо реализовать бота, который будет понимать обращение на естественном языке и определять категорию обращения

In [22]:
print("Loading modules...")
#<=============IMPORTS=============>
import pdfplumber
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
import pickle
import re
import numpy as np
import pymorphy2
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


#<=============Functions=============>
#loading model
def load_model(filename = "model.pkl"):
    with open(filename, 'rb') as file:
            model = pickle.load(file)
    return model

#predict
def make_pred(words, model, p_num=1):
    if p_num==1:
        pred = model.predict([words])
        return pred
    else:
        R = []
        allpreds = model.predict_proba([words]).reshape(-1) 
        for i in allpreds.argsort()[-p_num:][::-1]:
            R.append(model.classes_[i])
        return R
    
#words cleaning
def cleaning(words):
    print(">>> Всего слов: ", len(words.split()))
    morph = pymorphy2.MorphAnalyzer()
    a=len(words.split()) 
    words = re.sub('[^а-яА-ЯёЁ]', ' ', words).lower().strip()
    stopwords = ["тд","c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]
    words = ' '.join([word for word in words.split() if word not in (stopwords)])
    words = ' '.join([morph.parse(w)[0].normal_form for w in words.split()])
    print(">>> Убрано: ", a - len(words.split()))
    return words



# help
def helpp():
    print("Bot: Если у вас возникли какие-либо проблемы, убедитесь что все ваши файлы находятся в одной директории и вы везде указываете расширение файлов. В директрории должен обязательно содержаться файл model.pkl. Если вы считаете что вам выдается неточный результат знайте, что чем больше вы расскажете или напишите в вашем резюме о себе, тeм точнее будет результат. Eсли у вас есть обученная модель в формате pkl то вы можете заменить файл model.pkl в директории на свой \n\nВ боте содержaтся следующие функции load_model(filename = 'model.pkl'),pdf_text(filename), cleaning(words), make_pred(words, model, p_num=1), helpp()")
#<=============Functions=============>

print("Done")
print()



def main():
    while True:
        a = input("Bot: Для получения справки введите 'help' или 'no' если она вам не нужна \nUser: ")
        if a == "help":
            helpp()

        while True:
            a = input("Bot: Введите жалобу \nUser: ")

            print("Bot: Вы ввели: ", a)
            text = a
            break
        text = cleaning(text)
        print("Bot: loading model")
        model = load_model(filename="model.pkl")
        
        R = make_pred(text, model, p_num=1)
        print("<========RESULT========>")
        for i in R:
            print(i)
        print("<========RESULT========>")
            
        
        a = input("Bot: Хотите повторить?(y/n) \nUser: ")
        if a == "y":
            print("<====================================================>")
            continue
        else:
            break

main()

Loading modules...
Done

Bot: Для получения справки введите 'help' или 'no' если она вам не нужна 
User: no
Bot: Введите жалобу 
User: расклеины запрешенные объявления на этажах
Bot: Вы ввели:  расклеины запрешенные объявления на этажах
>>> Всего слов:  5
>>> Убрано:  1
Bot: loading model
<========RESULT========>
Содержание МКД
<========RESULT========>
Bot: Хотите повторить?(y/n) 
User: y
<====================================================>
Bot: Для получения справки введите 'help' или 'no' если она вам не нужна 
User: не убран мусор на улицах невозможно ходить
Bot: Введите жалобу 
User: не убран мусор на улицах невозможно ходить
Bot: Вы ввели:  не убран мусор на улицах невозможно ходить
>>> Всего слов:  7
>>> Убрано:  2
Bot: loading model
<========RESULT========>
Благоустройство
<========RESULT========>
Bot: Хотите повторить?(y/n) 
User: n


---

# Итог:
В ходе работы мы:
- разработали функции бота
- реализовали работу бота, которые обрабатывает естественный язык и определяет категорию обращения

In [23]:
class Cat:
    def __init__(self):
        self.meow = "yes"
        
    def showInfo(self):
        print(self.meow)

In [24]:
cat = Cat()

In [26]:
cat.showInfo()

yes
